In [5]:
import pandas as pd
import zipfile
import requests
from io import BytesIO
import os

In [6]:
url = 'https://download.inep.gov.br/microdados/microdados_enem_2019.zip'

In [12]:
os.makedirs('enem', exist_ok=True)
filebytes = BytesIO(requests.get(url, stream=True).content)

myzip = zipfile.ZipFile(filebytes)
myzip.extractall('enem')

In [13]:
#Biblioteca que auxilia carregar dados que excedem a 
#capacidade de memoria do computador, trabalhando no disco
import dask.dataframe as dd

In [18]:
data = dd.read_csv('./enem/DADOS/MICRODADOS_ENEM_2019.csv', sep = ';', decimal=',', encoding='latin1', dtype={'NU_IDADE':'float64'})

In [84]:
data = data.loc[data.SG_UF_RESIDENCIA == 'MG']

In [20]:
data = data.compute()

In [21]:
data.shape

(538896, 136)

In [22]:
data

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
4,190001237802,2019,3118601,Contagem,31,MG,22.0,F,1,1,...,B,A,A,B,A,A,C,B,B,B
14,190001237803,2019,3170206,Uberlândia,31,MG,30.0,F,1,2,...,A,A,A,B,A,A,B,A,A,A
25,190001237804,2019,3124104,Esmeraldas,31,MG,27.0,M,1,3,...,A,A,A,B,A,B,C,A,A,A
32,190001237805,2019,3118304,Conselheiro Lafaiete,31,MG,22.0,F,1,1,...,B,A,B,C,B,A,E,A,B,B
38,190001237806,2019,3162922,São Joaquim de Bicas,31,MG,23.0,F,1,4,...,B,A,A,B,B,A,C,A,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112744,190005766803,2019,3171204,Vespasiano,31,MG,23.0,M,1,2,...,B,A,A,C,A,B,D,A,B,B
112768,190005766804,2019,3170206,Uberlândia,31,MG,28.0,F,2,1,...,A,A,B,B,B,B,C,B,B,B
112779,190005766805,2019,3162500,São João del Rei,31,MG,20.0,M,1,4,...,B,A,B,C,B,B,C,B,B,B
112780,190005766806,2019,3158201,Santa Maria do Suaçuí,31,MG,18.0,F,1,3,...,A,A,A,B,A,A,B,A,A,A


In [30]:
data.to_csv('MG.csv')

In [31]:
data.columns

Index(['NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       ...
       'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024',
       'Q025'],
      dtype='object', length=136)

In [49]:
data['NU_NOTA_MT'] = data['NU_NOTA_MT'].apply(pd.to_numeric, errors='ignore')
data['NU_NOTA_CN'] = data['NU_NOTA_CN'].apply(pd.to_numeric, errors='ignore')
data['NU_NOTA_CH'] = data['NU_NOTA_CH'].apply(pd.to_numeric, errors='ignore')
data['NU_NOTA_LC'] = data['NU_NOTA_LC'].apply(pd.to_numeric, errors='ignore')

In [48]:
# média da nota em matemática de todos os alunos mineiros
data.NU_NOTA_MT.mean()

546.796207651218

In [50]:
#média da nota em Linguagens e Códigos de todos os alunos mineiros
data.NU_NOTA_LC.mean()

531.2155500475569

In [52]:
#média da nota em Ciências Humanas dos alunos do sexo FEMININO mineiros
data.loc[
    data.TP_SEXO == 'F', 'NU_NOTA_CH'
].mean()

515.1270983575348

In [53]:
#média da nota em Ciências Humanas dos alunos do sexo MASCULINO
data.loc[
    data.TP_SEXO == 'M', 'NU_NOTA_CH'
].mean()

529.6982704731556

In [60]:
#média da nota em Matemática dos alunos do sexo FEMININO que moram na cidade de Montes Claros
data.loc[
    (data.TP_SEXO == 'F')&(data.NO_MUNICIPIO_RESIDENCIA == 'Montes Claros'), 'NU_NOTA_MT'
].mean()

525.4776724249901

In [61]:
#média da nota em Matemática dos alunos do município de Sabará que possuem TV por assinatura na residência
data.loc[
    (data.Q021 == 'B')&(data.NO_MUNICIPIO_RESIDENCIA == 'Sabará'), 'NU_NOTA_MT'
].mean()

543.2927556818181

In [62]:
#média da nota em Ciências Humanas dos alunos mineiros que possuem dois fornos micro-ondas em casa
data.loc[
    data.Q016 == 'C', 'NU_NOTA_CH'
].mean()

557.2765986394558

In [63]:
#média em Matemática dos alunos mineiros cuja mãe completou a pós-graduação
data.loc[
    data.Q002 == 'G', 'NU_NOTA_MT'
].mean()

620.0070620709803

In [69]:
#média em Matemática dos alunos de Belo Horizonte e de Conselheiro Lafaiete
data.loc[
    (data.NO_MUNICIPIO_RESIDENCIA == 'Belo Horizonte')|(data.NO_MUNICIPIO_RESIDENCIA == 'Conselheiro Lafaiete'), 'NU_NOTA_MT'
].mean()

578.0392265100086

In [70]:
#média em Ciências Humanas dos alunos mineiros que moram sozinhos
data.loc[
    data.Q005== 1, 'NU_NOTA_CH'
].mean()

534.4573388609207

In [72]:
#média em Ciências Humanas 
#dos alunos mineiros cujo pai completou pós-graduação e possuem renda familiar entre R$ 8.982,01 e R$ 9.980,00
data.loc[
    (data.Q006 == 'M')&(data.Q001== 'G'), 'NU_NOTA_CH'
].mean()

586.7231663685158

In [74]:
#média em Matemática dos alunos do sexo Feminino que moram em Lavras e escolheram “Espanhol” como língua estrangeira
data.loc[
    (data.TP_SEXO == 'F')&(data.NO_MUNICIPIO_RESIDENCIA == 'Lavras')&(data.TP_LINGUA == 1), 'NU_NOTA_MT'
].mean()

510.80950782997775

In [75]:
#média em Matemática dos alunos do sexo Masculino que moram em Ouro Preto
data.loc[
    (data.TP_SEXO == 'M')&(data.NO_MUNICIPIO_RESIDENCIA == 'Ouro Preto'), 'NU_NOTA_MT'
].mean()

555.0832520325196

In [76]:
#nota média em Ciências Humanas dos alunos surdos
data.loc[
    data.IN_SURDEZ== 1, 'NU_NOTA_CH'
].mean()

435.38796296296283

In [83]:
#média em Matemática dos alunos do sexo FEMININO, 
#que moram em Belo Horizonte, Sabará, Nova Lima e Betim e possuem dislexia
data.loc[
    ((data.NO_MUNICIPIO_RESIDENCIA == 'Betim')|(data.NO_MUNICIPIO_RESIDENCIA == 'Nova Lima')|(data.NO_MUNICIPIO_RESIDENCIA == 'Belo Horizonte'))
    &(data.IN_DISLEXIA == 1)&(data.TP_SEXO == 'F'), 'NU_NOTA_MT'
].mean()

582.1935483870968